In [ ]:
import ivolatility as ivol
import pandas as pd
import os

ivolApiKey = os.getenv('API_KEY')
ivol.setLoginParams(apiKey=ivolApiKey)

# Let's now imagine that we are interested in extracting options on futures from the previous example No.5
getFutData = ivol.setMethod('/futures/eod/futures-info')
futDataRaw = getFutData(date='2023-11-01',underlyingName='E%Mini%S%P%500%')

futuresNeed = getFutData(date='2023-11-01',rootSymbol='ES', MIC = 'XCME')

# Let's now find the prices for these futures for period from '2023-11-01' to '2023-11-10'
geFutPriceData = ivol.setMethod('/futures/single-futures')


finalFutData = pd.DataFrame()
for futSymbol in futuresNeed['FutSymbol']:  # Corrected column name
    futPriceData = geFutPriceData(symbol=futSymbol, from_='2023-11-01', to='2023-11-10')
    finalFutData = pd.concat([finalFutData, futPriceData], axis=0)
    
'''
Creating a list of required futures (take from the previous example) 
with expiration dates between "2024-08-21" and "2024-10-21"
'''

myFutList = finalFutData['Future symbol'][(finalFutData['Future expiration'] > '2024-08-21') & (finalFutData['Future expiration'] < '2024-10-21')].unique()

futOptsList = pd.DataFrame()

'''
Suppose we need call options on ES futures also with expiration dates between "2024-08-21" and "2024-10-21", 
a strike range between 3800 and 3900 and on the date "2024-08-21"
'''

# The process of extracting option tickers based on chosen parameters
getFutOptsData = ivol.setMethod('/futures/eod/option-series-on-date')
for futSymbol in myFutList:
  futOptsNeed = getFutOptsData(date='2024-08-01', futuresSymbol=futSymbol, expFrom='2024-08-21', expTo='2024-10-21', strikeFrom=3800, strikeTo=3900, callPut='C')
  futOptsList = pd.concat([futOptsList, futOptsNeed], axis=0)

finalOptsCalcData = pd.DataFrame()

# Getting futures options pricing data with implied volatilities and Greeks
getFutOptsCalcData = ivol.setMethod('/futures/eod/single-future-option-rawiv')
for futOptSymbol in futOptsList['opt_symbol']:
  futOptsCalcData = getFutOptsCalcData(symbol=futOptSymbol, from_='2024-08-21', to='2024-09-01')
  finalOptsCalcData = pd.concat([finalOptsCalcData, futOptsCalcData], axis=0)

finalOptsCalcData

!!!!


,date,symbol,option root symbol,Future symbol,exchange,Future expiration,Adjusted close,expiration,strike,Call/Put,style,ask,bid,Settlement price,volume,Open interest,iv,iv_interpolated,vega,delta,gamma,theta
0,2024-08-21,ES,E3C,ES/24U.CM,CME,2024-09-20,5641.50,2024-08-21,3800.0,C,E,0.00,0.0,1841.39,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2024-08-21,ES,E3C,ES/24U.CM,CME,2024-09-20,5641.50,2024-08-21,3850.0,C,E,0.00,0.0,1791.39,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2024-08-21,ES,E3C,ES/24U.CM,CME,2024-09-20,5641.50,2024-08-21,3900.0,C,E,0.00,0.0,1741.39,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
0,2024-08-21,ES,E4D,ES/24U.CM,CME,2024-09-20,5641.50,2024-08-22,3800.0,C,E,1844.75,1837.5,1841.25,0,0,-1.000000,0.185078,0.000000,1.000000,0.000000,0.000000
1,2024-08-22,ES,E4D,ES/24U.CM,CME,2024-09-20,5594.00,2024-08-22,3800.0,C,E,0.00,0.0,1794.11,0,0,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2024-08-26,ES,ES,ES/24U.CM,CME,2024-09-20,5637.00,2024-09-20,3900.0,C,A,0.00,0.0,1737.00,0,0,0.689340,0.689340,0.604061,0.983501,0.000041,-0.577342
4,2024-08-27,ES,ES,ES/24U.CM,CME,2024-09-20,5644.75,2024-09-20,3900.0,C,A,0.00,0.0,1744.75,0,0,0.702258,0.702258,0.578643,0.983957,0.000040,-0.589199
5,2024-08-28,ES,ES,ES/24U.CM,CME,2024-09-20,5610.25,2024-09-20,3900.0,C,A,0.00,0.0,1710.25,0,0,0.700890,0.700890,0.549524,0.984406,0.000040,-0.585149
6,2024-08-29,ES,ES,ES/24U.CM,CME,2024-09-20,5610.00,2024-09-20,3900.0,C,A,0.00,0.0,1710.00,0,0,0.710616,0.710616,0.518644,0.985042,0.000039,-0.587928
